In [1]:
import LookUpTable as lut
import pandas as pd
import re
import random
import Levenshtein

In [2]:
SamantarDatasetPath = r'./SamantarDatasetRaw.txt'
webscrapedDatasetPath = r'./web_scraped_data_raw.csv'

In [3]:
def clean_telugu_text(text):
    # Remove non-Telugu characters except whitespace
    cleaned = re.sub(r'[^\u0C00-\u0C7F\s]', '', text)
    # Replace multiple spaces with a single space
    cleaned = re.sub(r'\s+', ' ', cleaned)
    # Strip unnecessary leading/trailing whitespace
    cleaned = cleaned.strip()
    
    return cleaned

In [4]:
def get_levenstein(src, tar):
    return Levenshtein.distance(src, tar)

In [56]:
def substitute_consonants(text, error_percentage):
    characters = list(text)
    max_errors = int(len(characters) * error_percentage)
    error_count = 0
    tried_indices = set()
    
    start_length = len(characters)

    while error_count < max_errors and len(tried_indices) < len(characters):
        idx = random.randint(0, len(characters) - 1)
        if idx in tried_indices:
            continue
        tried_indices.add(idx)

        current_char = characters[idx]

        # Skip if character is not a consonant present the lookup table
        if current_char not in lut.SimpleCharacterReplacements.keys():
            continue

        # Skip if previous character is a virama (్)
        if (idx > 0 and characters[idx - 1] == '్') or (characters[idx]=="య" and idx < len(characters) - 1 and characters[idx + 1] == "ి"):
            continue

        # Get replacement options and choose one randomly
        replacements = lut.SimpleCharacterReplacements[current_char]
        replacement = random.choice(replacements)

        characters[idx] = replacement
        error_count += 1

    end_length = len(characters)

    avg_error = 0
    for (actual, error) in zip(text.split(" "), ''.join(characters).split(" ")):
        avg_error += get_levenstein(actual, error)
        
    AverageErrorPerWord = round(avg_error / len(text.split(" ")), 2)
    PercentageErrorInduced = round((error_count / start_length), 2) *100

    return ''.join(characters), PercentageErrorInduced, AverageErrorPerWord

In [57]:
def substitute_stacked_diacritics(text, error_percentage):
    
    telugu_consonants = ['క', 'ఖ', 'గ', 'ఘ', 'ఙ','చ', 'ఛ', 'జ', 'ఝ', 'ఞ',
                         'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న',
                        'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ల', 'వ',
                        'శ', 'ష', 'స', 'హ', 'ళ', 'క్ష', 'ఱ']

    characters = list(text)
    max_errors = int(len(characters) * error_percentage)
    error_count = 0
    tried_indices = set()
    
    start_length = len(characters)

    while error_count < max_errors and len(tried_indices) < len(characters):
        idx = random.randint(0, len(characters) - 1)
        if idx in tried_indices:
            continue
        tried_indices.add(idx)

        current_char = characters[idx]

        # Skip if character is not a consonant present the lookup table
        if current_char not in lut.StackingReplacements.keys() and current_char not in lut.Diacritic_replacements.keys():
            continue

        # Check if previous character is a virama (్) only then we can replace the current character
        if ((idx > 2 and characters[idx - 1] == '్' and characters[idx - 2] in telugu_consonants) or
            (current_char in lut.Diacritic_replacements.keys() and characters[idx - 1] in telugu_consonants)):
            # Get replacement options and choose one randomly
            if(current_char in lut.Diacritic_replacements.keys()):
                replacements = lut.Diacritic_replacements[current_char]
            else:
                replacements = lut.StackingReplacements[current_char]
    
            replacement = random.choice(replacements)

            characters[idx] = replacement
            error_count += 1

    avg_error = 0
    for (actual, error) in zip(text.split(" "), ''.join(characters).split(" ")):
        avg_error += get_levenstein(actual, error)
        
    AverageErrorPerWord = round(avg_error / len(text.split(" ")), 2)
    PercentageErrorInduced = round((error_count / start_length), 2)

    return ''.join(characters), PercentageErrorInduced, AverageErrorPerWord

### Dealing with Samantar Dataset

In [78]:
with open(SamantarDatasetPath, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file.readlines()]

df = pd.DataFrame(lines, columns=["OriginalText"])

In [79]:
df["OriginalText"] = df["OriginalText"].apply(clean_telugu_text)

In [80]:
random_text = df.sample(1).iloc[0]['OriginalText']
print("Original Text : ", random_text)
print(list(random_text))
print("Error Induced Text : ", substitute_consonants(random_text, 0.1)[0])
print("Average Error per word : ", substitute_consonants(random_text, 0.1)[1])

Original Text :  మమతా బెనర్జీ ఫైల్ ఇమేజ్
['మ', 'మ', 'త', 'ా', ' ', 'బ', 'ె', 'న', 'ర', '్', 'జ', 'ీ', ' ', 'ఫ', 'ై', 'ల', '్', ' ', 'ఇ', 'మ', 'ే', 'జ', '్']
Error Induced Text :  యమతా భెనర్జీ ఫైల్ ఇమేజ్
Average Error per word :  9.0


In [81]:
# df[["15%_ErrorInducedText", "Percentage_ErrorInduced_15%", "AverageErrorPerWord_15%"]] = df["OriginalText"].apply(lambda x: substitute_consonants(x, 0.15)).apply(pd.Series)
# df[["25%_ErrorInducedText", "Percentage_ErrorInduced_25%", "AverageErrorPerWord_25%"]] = df["OriginalText"].apply(lambda x: substitute_consonants(x, 0.25)).apply(pd.Series)
# df[["35%_ErrorInducedText", "Percentage_ErrorInduced_35%", "AverageErrorPerWord_35%"]] = df["OriginalText"].apply(lambda x: substitute_consonants(x, 0.35)).apply(pd.Series)
# df[["50%_ErrorInducedText", "Percentage_ErrorInduced_50%", "AverageErrorPerWord_50%"]] = df["OriginalText"].apply(lambda x: substitute_consonants(x, 0.5)).apply(pd.Series)
# df.to_csv('./SamantarDatasetWithDirectConsonantSubstitutions.csv', index=False, encoding='utf-8-sig')

In [82]:
df[["15%_ErrorInducedText", "Percentage_ErrorInduced_15%", "AverageErrorPerWord_15%"]] = df["OriginalText"].apply(lambda x: substitute_stacked_diacritics(x, 0.15)).apply(pd.Series)
df[["25%_ErrorInducedText", "Percentage_ErrorInduced_25%", "AverageErrorPerWord_25%"]] = df["OriginalText"].apply(lambda x: substitute_stacked_diacritics(x, 0.25)).apply(pd.Series)
df[["35%_ErrorInducedText", "Percentage_ErrorInduced_35%", "AverageErrorPerWord_35%"]] = df["OriginalText"].apply(lambda x: substitute_stacked_diacritics(x, 0.35)).apply(pd.Series)
df[["50%_ErrorInducedText", "Percentage_ErrorInduced_50%", "AverageErrorPerWord_50%"]] = df["OriginalText"].apply(lambda x: substitute_stacked_diacritics(x, 0.5)).apply(pd.Series)
df.to_csv('./SamantarDatasetWithStackedSubstitutions.csv', index=False, encoding='utf-8-sig')

In [63]:
# Model needs to learn from more context and accordingly correct the text
# substitute_stacked_diacritics("శంకరాచార్యులు అద్వైత సిద్ధాంతాన్ని ప్రవేశపెట్టారు", 0.15)

### Dealing with Web Scraped Dataset

In [83]:
scraped_df = pd.read_csv(webscrapedDatasetPath, encoding='utf-8-sig')

In [84]:
Q1 = scraped_df['word_count'].quantile(0.25)
Q3 = scraped_df['word_count'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = scraped_df[(scraped_df['word_count'] < lower_bound) | (scraped_df['word_count'] > upper_bound)]

print("Maximum word count in the dataset:", scraped_df['word_count'].max())
print("Min word count in the dataset:", scraped_df['word_count'].min())

print("Number of outliers:", len(outliers))

Maximum word count in the dataset: 500
Min word count in the dataset: 13
Number of outliers: 0


In [85]:
# since there are no outliers, split each sentence into 3 sentences of length total number of words/3 = 500/3 ~ 160
max_sentence_length = 100 # max number of words in a sentence
split_sentences_list = []
for index, row in scraped_df.iterrows():
    text = row['content']
    words = text.split()
    for i in range(0, len(words), max_sentence_length):
        new_sentence = ' '.join(words[i:i + max_sentence_length])
        split_sentences_list.append(new_sentence)

# remove the empty strings from the list
split_sentences_list = [sentence for sentence in split_sentences_list if sentence.strip() != '']

# filter out the sentence with sentence length < 20 words
split_sentences_list = [sentence for sentence in split_sentences_list if len(sentence.split()) >= 20]

In [86]:
print("Number of sentences after splitting:", len(split_sentences_list))
print("Max sentence length after splitting:", max([len(sentence.split()) for sentence in split_sentences_list]))
print("Min sentence length after splitting:", min([len(sentence.split()) for sentence in split_sentences_list]))
print("Average sentence length after splitting:", sum([len(sentence.split()) for sentence in split_sentences_list]) / len(split_sentences_list))

Number of sentences after splitting: 3240
Max sentence length after splitting: 100
Min sentence length after splitting: 20
Average sentence length after splitting: 89.22901234567901


In [87]:
scraped_data_df = pd.DataFrame(split_sentences_list, columns=["OriginalText"])
scraped_data_df["OriginalText"] = scraped_data_df["OriginalText"].apply(clean_telugu_text)

In [88]:
# scraped_data_df[["15%_ErrorInducedText", "Percentage_ErrorInduced_15%", "AverageErrorPerWord_15%"]] = scraped_data_df["OriginalText"].apply(lambda x: substitute_consonants(x, 0.15)).apply(pd.Series)
# scraped_data_df[["25%_ErrorInducedText", "Percentage_ErrorInduced_25%", "AverageErrorPerWord_25%"]] = scraped_data_df["OriginalText"].apply(lambda x: substitute_consonants(x, 0.25)).apply(pd.Series)
# scraped_data_df[["35%_ErrorInducedText", "Percentage_ErrorInduced_35%", "AverageErrorPerWord_35%"]] = scraped_data_df["OriginalText"].apply(lambda x: substitute_consonants(x, 0.35)).apply(pd.Series)
# scraped_data_df[["50%_ErrorInducedText", "Percentage_ErrorInduced_50%", "AverageErrorPerWord_50%"]] = scraped_data_df["OriginalText"].apply(lambda x: substitute_consonants(x, 0.5)).apply(pd.Series)
# scraped_data_df.to_csv('./WebScrapedDatasetWithDirectConsonantSubstitutions.csv', index=False, encoding='utf-8-sig')

In [89]:
scraped_data_df[["15%_ErrorInducedText", "Percentage_ErrorInduced_15%", "AverageErrorPerWord_15%"]] = scraped_data_df["OriginalText"].apply(lambda x: substitute_stacked_diacritics(x, 0.15)).apply(pd.Series)
scraped_data_df[["25%_ErrorInducedText", "Percentage_ErrorInduced_25%", "AverageErrorPerWord_25%"]] = scraped_data_df["OriginalText"].apply(lambda x: substitute_stacked_diacritics(x, 0.25)).apply(pd.Series)
scraped_data_df[["35%_ErrorInducedText", "Percentage_ErrorInduced_35%", "AverageErrorPerWord_35%"]] = scraped_data_df["OriginalText"].apply(lambda x: substitute_stacked_diacritics(x, 0.35)).apply(pd.Series)
scraped_data_df[["50%_ErrorInducedText", "Percentage_ErrorInduced_50%", "AverageErrorPerWord_50%"]] = scraped_data_df["OriginalText"].apply(lambda x: substitute_stacked_diacritics(x, 0.5)).apply(pd.Series)
scraped_data_df.to_csv('./WebScrapedDatasetWithStackedSubstitutions.csv', index=False, encoding='utf-8-sig')